# Tarea 3 - Inteligencia artificial

'/home/ubuntu/inteligencia_artificial/Tareas/Tarea 3'

## Item 1
Importo dataset (10 puntos)
Fuente: https://www.kaggle.com/datasets/ayessa/salary-prediction-classification

In [68]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
dataset = pd.read_csv("./dataset/train22k.csv")
print(dataset.head())
dataset.shape

   age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital-status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  salary  
0          2174             0              40   United-States   <=50

(21999, 15)

## Item 2
Entrene el algoritmo KNN sobre los datos seleccionados y aplique predicciones con algún (sub)conjunto DISJUNTO de prueba. Adicionalmente, utilice 3 métricas (a su elección) de error y compárelas entre sí. (15 ptos)

In [95]:
from sklearn.neighbors import KNeighborsClassifier
""" el siguiente es para asociar strings a numeros """
from sklearn.preprocessing import LabelEncoder


#print(type(dataset.iloc[0,1]))
#print(isinstance(dataset.iloc[0,1], str))
#print(dataset.head())
""" Declaro el encoder de string a numeros """
encoder = LabelEncoder()

def column_string_to_int(columnas, dataset_validation):
     """ For para iterar el dataset en columnas"""
     for i in range(columnas):
          """  
          En realidad el dataset es un dataframe de pandas, este tiene propiedades para
          acceder a él. 'iloc' sirve para acceder a él como una matriz con indices
          de fila y columna [x,y].

          Con isisntance() puedo ver si la primera fila de datos en cada columna es un string,
          ya que en estos casos debo aplicar la conversion a números """
          if isinstance(dataset_validation.iloc[0,i], str):
               """
               Con dataset.columns[i] obtengo el nombre de la columna,
               luego convierto esa columna completa en números """
               string = dataset_validation.columns[i]
               dataset_validation[string] = encoder.fit_transform(dataset_validation[string])

def best_k(x_train, y_train, x_test, y_test, max_k):
     best_k = 0
     best_score = 0
     for i in range(max_k):
          i = i+1
          knn = KNeighborsClassifier(n_neighbors=i)
          knn.fit(x_train, y_train)
          score = knn.score(x_test, y_test)
          if best_score < score:
               best_score = score
               best_k = i
          print("Procesando actualmente K=",i,", mejor k=",best_k, ", mayor precision: ", best_score)

#Considerar que se debe aplicar tanto para el train como el test
def eliminar_columna(x, columna):
     x = np.delete(x, columna, 1)
     return x

def contar_y(y, nombre_dataset=""):
     #si es 0 es <=50k, es decir, menor que 50k
     counter_menor = 0
     counter_mayor = 0
     for i in range(len(y)):
          if y[i] == 0:
               counter_menor += 1
          else:
               counter_mayor += 1
     print("y", nombre_dataset," <=50k: ", counter_menor, " corresponde al: ", counter_menor*100 / len(y), "%" )
     print("y", nombre_dataset,"  >50k: ", counter_mayor, " corresponde al: ", counter_mayor*100 / len(y), "%")

column_string_to_int(15, dataset)

x = dataset.iloc[:, :14].values

y = dataset['salary'].values


#Para eliminar una columna: 
""" print("x:", x)
print("Cantidad de columnas: ", len(x[0]))
x = np.delete(x,7,1)
print("Cantidad de columnas: ", len(x[0])) """

knn = KNeighborsClassifier(n_neighbors=17)

#x = eliminar_columna(x, 2)

#print(x)
knn.fit(x, y)
#Realizo una predicción

dataset_validation = pd.read_csv("./dataset/validation.csv")
column_string_to_int(15, dataset_validation)

x_validation = dataset_validation.iloc[:, :14].values
y_validation = dataset_validation['salary'].values

#x_validation = eliminar_columna(x_validation, 2)

#knn.predict(x_validation[1].reshape(1,-1))[0]
#best_k(x,y,x_validation,y_validation,4)

#Predice los resultados "y" respecto a la matriz "x" de validación (x_validation)
y_pred = knn.predict(x_validation)

score_train = knn.score(x,y)
score_validation = knn.score(x_validation,y_validation)
""" print("Score_train: ", score_train)
print("score_validation: ", score_validation) """

from sklearn import metrics

print("accuracy_score:", metrics.accuracy_score(y_validation, y_pred))
print("hamming_loss:", metrics.hamming_loss(y_validation, y_pred))
print("Average precision score: ", metrics.average_precision_score(y_validation, y_pred))
print("Confusion matrix:\n", metrics.confusion_matrix(y_validation, y_pred))

matrix_porcentaje = metrics.confusion_matrix(y_validation, y_pred)

for i in range(len(matrix_porcentaje)):
     for j in range(len(matrix_porcentaje[i])):
          matrix_porcentaje[i][j] = matrix_porcentaje[i][j]*100/ len(y_validation)
          
print("Porcentajes en confusion matrix: \n", matrix_porcentaje)



contar_y(y, "train")
contar_y(y_validation, "validation")


accuracy_score: 0.7954932777882977
hamming_loss: 0.20450672221170232
Average precision score:  0.3707464255958146
Confusion matrix:
 [[7846  115]
 [2045  556]]
Porcentajes en confusion matrix: 
 [[74  1]
 [19  5]]
y train  <=50k:  16759  corresponde al:  76.18073548797673 %
y train   >50k:  5240  corresponde al:  23.819264512023274 %
y validation  <=50k:  7961  corresponde al:  75.37398220034085 %
y validation   >50k:  2601  corresponde al:  24.626017799659156 %


# Análisis

Se seleccionaron las siguientes tres  metricas de error:
1. **Accuracy score:** Corresponde a la probabilidad de predicciones correctas del modelo ya entrenado. Para ello le damos como parámetros una matriz $x$ que corresponde a los datos en los que se basa el modelo (Son nuevos datos distintos a los de entrenamiento, aunque también se puede ver con los datos de entrenamiento) para generar predicciones y un parámetro $y$ que corresponde a los resultados correctos. Así el output de esta función es:

     (total_de_resultados_correctamente_acertados) / (total_de_resultados_correctos)

  Se aprecia que las predicciones obtenidas tienen un acierto del 79% aproximadamente, lo cual si bien tiene una tendencia a dar una respuesta correcta, sigue con un porcentaje de error considerable entorno al 21%. Por lo cual no sería una predicción tan confiable, aunque esto es relativo al contexto en el que se necesite.

2. **Average precision score:** Corresponde a las nociones que tiene de precisión que se pueden aplicar a cada etiqueta de forma independiente , lo cual entrega una aproximacion del resultado obtenido por y_pred.
3. **Matriz de confusión:** La matriz de confusión del inglés Confussion matrix, es una matriz que entrega una forma de visualización de fácil interpretación. Esto se puede explicar de forma más fácil con un diagrama:

<img src="./img/matriz_confusion.jpg" alt="Matriz de confusion" width="75%" style="
  display: block; 
  margin-left: auto;
  margin-right: auto;" />

En donde los verdaderos positivos son los que tienen tanto en la predicción, como en la respuesta real un valor de 0 (<=50k). Los verdaderos negativos son los que en ambos tienen un valor de 1 (>50k). Los falsos positivos son los que en la predicción tienen un valor de 0 (<=50k), pero en la respuesta real un valor de 1 (>50k), por lo que son predicciones erróneas. Los falsos negativos son los que en la predicción son 1 (>50k), pero en el real 0 (<=50k).

Al ver la impresón de la confusion matrix con los datos en porcentajes, es aún más fácil realizar una interpretación. Asi se puede observar que los verdaderos positivos (<=50k) son el 74% siendo por lejos la mayor cantidad de resultados. Seguido por un tambien relevante 19% de falsos positivos (<=50k). Teniendo entre estos dos el 94%. 

## Item 3
Entrene el algoritmo de Regresión Lineal ahora con los mismos datos seleccionados, aplicando también predicciones sobre el mismo (sub)conjunto DISJUNTO de prueba de la parte anterior. Elija alguna métrica de error de entre las tres de la parte anterior y compare los errores, dando una explicación de qué sucede. (15 ptos)

In [70]:
# Imports necesarios
import numpy as np
import pandas as pd
#import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

#cargamos los datos de entrada
data = pd.read_csv("./dataset/train22k.csv")
#veamos cuantas dimensiones y registros contiene
data


from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()



column_string_to_int(15, data)

# Asignamos nuestra variable de entrada X para entrenamiento y las etiquetas Y.

X_train = data.iloc[:, :14].values
#Nocumple = db[db.salary.str.contains('<=50K')]

y_train = data['salary'].values

# Creamos el objeto de Regresión Linear
regr = linear_model.LinearRegression()
 
# Entrenamos nuestro modelo
regr.fit(X_train, y_train)
y_pred = regr.predict(X_train)
#llamamos otro dataset...
data_validation = pd.read_csv("./dataset/validation.csv")

column_string_to_int(15, data_validation)

#print(dataset_validation.head())
X_train_validation = data_validation.iloc[:, :14].values
y_train_validation = data_validation['salary'].values
y_pred1 = regr.predict(X_train_validation)
""" Los datos de salario <=50k son 0 y los >50 son 1 """

#metricas score...

from sklearn import metrics

score_train1 = metrics.r2_score(y_train, y_pred)
score_validation1 = metrics.r2_score(y_train_validation, y_pred1)

print("Score_train: ", score_train1)
print("score_validation: ", score_validation1)

print(metrics.max_error(y_train_validation, y_pred1))
#listoco...




Score_train:  0.25645056075503647
score_validation:  0.2729230256309212
1.2753269315778546


## Item 4
Tome su mismo dataset y entrene dos algoritmos de clasificación: Gaussian Mixture Model y DBSCAN. Evalúe estos algoritmos sobre el mismo (sub)conjunto DISJUNTO de las partes 2 y 3. Analice la diferencia en los resultados entre los algoritmos (10 ptos). Asimismo, indique y aporte evidencia en la diferencia fundamental (clustering vs. estimación de densidad) que existe entre ambos algoritmos (10 ptos).

In [71]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=4, random_state=123).fit(x)
#gm.means_
#c = gm.predict(x)

print(gm.predict(x))
print(gm.predict(x_validation))
print(gm.score(x))
print(gm.weights_)

[1 3 3 ... 3 3 1]
[3 0 3 ... 3 3 0]
-28.94735451565134
[0.01609181 0.07073507 0.04250194 0.87067118]


In [72]:
from sklearn.cluster import DBSCAN

cluster = DBSCAN(eps=2 , min_samples=10)

In [73]:
data.shape

(21999, 15)

In [74]:
data.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12]].values

array([[    39,      7,  77516, ...,   2174,      0,     40],
       [    50,      6,  83311, ...,      0,      0,     13],
       [    38,      4, 215646, ...,      0,      0,     40],
       ...,
       [    19,      4, 272063, ...,      0,      0,     35],
       [    34,      4, 169564, ...,      0,      0,     40],
       [    56,      4, 188856, ...,   8614,      0,     55]])